In [ ]:
# Import necessary modules
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from collections import Counter
import string
import pandas as pd
import matplotlib.pyplot as plt

# Load the spacy model and add spacytextblob for sentiment analysis
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

# Load dataset
reviews = pd.read_csv('insert relative filename.csv') # Add the proper filename

# Remove stopwords, punctuation, and convert to lowercase
def preprocess_reviews(reviews):
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    filtered_tokens = []

    for sentence in reviews['relevant column']: # Select the relevant column
        doc = nlp(sentence)
        tokens = [token.text.lower() for token in doc if not token.is_stop and token.text not in string.punctuation]
        filtered_tokens.append(tokens)
    
    return filtered_tokens

# Define a function that retrieves the most frequent adjectives in reviews
def filter_adjectives(tokens):
    adjectives = []
    for token in tokens:
        doc = nlp(token)
        for token in doc:
            if token.pos_ == 'ADJ': # Check if token is an adjective
                if doc._.polarity < 0:  # Check if token is a 'positive' adjective
                    adjectives.append(token.text)
    return adjectives

# Define a function that retrieves adjectives with negative polarities
def filter_adjectives_with_negative_polarity(tokens):
    negative_adjectives = []
    for token in tokens:
        doc = nlp(token)
        for token in doc:
            if token.pos_ == 'ADJ':  
                if doc._.polarity > 0:  # Check if token is a 'negative' adjective
                    negative_adjectives.append(token.text)
    return negative_adjectives

# Plot histograms
def plot_adjective_histograms(positive_adjectives, negative_adjectives):
    # Calculate adjective frequencies
    pos_adj_freq = Counter(positive_adjectives)
    neg_adj_freq = Counter(negative_adjectives)

    # Get the top 10 positive and negative adjectives
    top_positive_adjectives = pos_adj_freq.most_common(10)
    top_negative_adjectives = neg_adj_freq.most_common(10)

    # Unpack the adjectives and counts
    pos_adjectives, pos_counts = zip(*top_positive_adjectives)
    neg_adjectives, neg_counts = zip(*top_negative_adjectives)

    # Plot histograms
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.bar(pos_adjectives, pos_counts, color='green')
    plt.xlabel('Positive Adjectives')
    plt.ylabel('Frequency')
    plt.title('Top Positive Adjectives')
    plt.xticks(rotation=45)

    plt.subplot(1, 2, 2)
    plt.bar(neg_adjectives, neg_counts, color='red')
    plt.xlabel('Negative Adjectives')
    plt.ylabel('Frequency')
    plt.title('Top Negative Adjectives')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

# Main function to run all steps
def main():
    # Preprocess the reviews
    tokens = preprocess_reviews(reviews)

    # Flatten the list of lists into a single list of tokens
    flat_tokens = [token for sublist in tokens for token in sublist]

    # Extract and analyze positive adjectives
    positive_adjectives = filter_adjectives(flat_tokens)

    # Extract and analyze negative adjectives
    negative_adjectives = filter_adjectives_with_negative_polarity(flat_tokens)

    # Calculate adjective frequencies
    adj_freq_positive = Counter(positive_adjectives)
    adj_freq_negative = Counter(negative_adjectives)

    # Rank adjectives by frequency to extract top 10
    top_positive_adjectives = adj_freq_positive.most_common(10)
    top_negative_adjectives = adj_freq_negative.most_common(10)

    # Print the top positive adjectives
    print("\033[1mTop Positive Adjectives:\033[0m")
    for adj, freq in top_positive_adjectives:
        print(f"{adj}: {freq}")

    # Print the top negative adjectives
    print("\033[1mTop Negative Adjectives:\033[0m")
    for adj, freq in top_negative_adjectives:
        print(f"{adj}: {freq}")

    # Plot histograms for top positive and negative adjectives
    plot_adjective_histograms(positive_adjectives, negative_adjectives)

# Execute the main function
if __name__ == "__main__":
    main()
